In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint
m = wt.watertap_setup(dynamic = False)


wt.case_study_trains.train = {"case_study": "hrsd",
                             "reference": "nawi",
                             "scenario": "baseline"}

wt.case_study_trains.source_water = {"case_study": "hrsd", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": "hrsd_municipal"}

m = wt.case_study_trains.get_case_study(m=m) # flow is set as case study flow unless defined.




hrsd
------- Adding Unit Processes -------
sw_onshore_intake
coagulant_addition
coag_and_floc
sedimentation
ozone_aop
fixed_bed_gravity_basin
gac_pressure_vessel
uv_aop
chlorination
treated_storage
21600.0
recharge_pump_well
sludge_tank
-------------------------------------
adding source
----- Connecting Unit Processes -----
hrsd_municipal ToUnitName --> sw_onshore_intake
sw_onshore_intake ToUnitName --> coagulant_addition
coagulant_addition ToUnitName --> coag_and_floc
coag_and_floc ToUnitName --> sedimentation
sedimentation ToUnitName --> ozone_aop
ozone_aop ToUnitName --> fixed_bed_gravity_basin
fixed_bed_gravity_basin ToUnitName --> gac_pressure_vessel
gac_pressure_vessel ToUnitName --> uv_aop
gac_pressure_vessel ToUnitName --> sludge_tank
uv_aop ToUnitName --> chlorination
chlorination ToUnitName --> treated_storage
treated_storage ToUnitName --> recharge_pump_well
-------------------------------------


In [2]:
wt.display.show_train2(model_name=m)

In [3]:
# # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("unfixing feed presure and area for", key)
        
wt.run_water_tap(m = m, objective=True)

# prints RO results
print("optimal ro area and pressures:")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

# RESET PRESSURE TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        if "feed_pressure" in m.fs.pfd_dict[key]["Parameter"]:
            if m.fs.pfd_dict[key]["Parameter"]["type"] == "pass":
                getattr(m.fs, key).feed.pressure.fix(m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])
                print("setting feed presure for", key, "to -->", m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])

wt.run_water_tap(m = m, objective=True, print_model_results="summary", skip_small = True)

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

degrees_of_freedom: 0
WaterTAP3 solution optimal
optimal ro area and pressures:
degrees_of_freedom: 0
WaterTAP3 solution optimal
fs.sw_onshore_intake
total_cap_investment: 1.091405800373164
----------------------------------------------------------------------
fs.coagulant_addition
total_cap_investment: 0.859865843461623
----------------------------------------------------------------------
fs.coag_and_floc
total_cap_investment: 3.437239278286118
----------------------------------------------------------------------
fs.sedimentation
total_cap_investment: 0.5827579419005313
----------------------------------------------------------------------
fs.ozone_aop
total_cap_investment: 17.49928939833537
----------------------------------------------------------------------
fs.fixed_bed_gravity_basin
total_cap_investment: 13.363784432168028
----------------------------------------------------------------------
fs.gac_pressure_vessel
total_cap_investment: 13.4394246517668
------------------------

In [4]:
# set cap utilization factor
m.fs.costing_param.plant_cap_utilization = 1

In [5]:
wt.run_water_tap(m = m, objective=True, print_model_results="summary", skip_small = True)
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

degrees_of_freedom: 0
WaterTAP3 solution optimal
fs.sw_onshore_intake
total_cap_investment: 1.091405800373164
----------------------------------------------------------------------
fs.coagulant_addition
total_cap_investment: 0.859865843461623
----------------------------------------------------------------------
fs.coag_and_floc
total_cap_investment: 3.437239278286118
----------------------------------------------------------------------
fs.sedimentation
total_cap_investment: 0.5827579419005313
----------------------------------------------------------------------
fs.ozone_aop
total_cap_investment: 17.49928939833537
----------------------------------------------------------------------
fs.fixed_bed_gravity_basin
total_cap_investment: 13.363784432168028
----------------------------------------------------------------------
fs.gac_pressure_vessel
total_cap_investment: 13.4394246517668
----------------------------------------------------------------------
fs.uv_aop
total_cap_investment: 4

In [6]:
# RESET AREA TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).membrane_area.fix(getattr(m.fs, key).membrane_area[0]())

In [7]:
# Readjust recovery constraint and deactivate objective constraint
m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery >= 0)
m.fs.objective_function.deactivate()

In [8]:
# NOW RUN AS SIMULATION
wt.run_water_tap(m = m, objective=False, print_model_results="summary", skip_small = True)

degrees_of_freedom: 0
WaterTAP3 solution optimal
fs.sw_onshore_intake
total_cap_investment: 1.091405800373164
----------------------------------------------------------------------
fs.coagulant_addition
total_cap_investment: 0.859865843461623
----------------------------------------------------------------------
fs.coag_and_floc
total_cap_investment: 3.437239278286118
----------------------------------------------------------------------
fs.sedimentation
total_cap_investment: 0.5827579419005313
----------------------------------------------------------------------
fs.ozone_aop
total_cap_investment: 17.49928939833537
----------------------------------------------------------------------
fs.fixed_bed_gravity_basin
total_cap_investment: 13.363784432168028
----------------------------------------------------------------------
fs.gac_pressure_vessel
total_cap_investment: 13.4394246517668
----------------------------------------------------------------------
fs.uv_aop
total_cap_investment: 4

In [9]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m = m, case_study = wt.case_study_trains.source_water["case_study"], 
                                scenario = wt.case_study_trains.source_water["scenario"])